In [2]:
import dicom as dcm
import SimpleITK as sitk
import numpy as np 
import array
from numpy import unravel_index
import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interact
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice], cmap=plt.cm.jet)
    interact(plot_image, myslice=(0, im.shape[0]))
    return;

In [3]:

#print(rp)
#rp=dcm.read_file('../interplay/output/RTplan_Resampled/1D/RP.test_orientation.dcm')
#print(rp)

# GATE output and RT dose DICOM

In [85]:
#Get the rd raw created  and gate image

#name='interplay_1D_FFF_PTV25_A25T7_dose'
#rp=dcm.read_file('../interplay/output/RTplan_Resampled/2D/2D_SIN_ITV.dcm')
rp=dcm.read_file('../interplay/output/new/RTplan/RP_2D_CUR_ITV_5_5.dcm')
                 
name='interplay_2D_CUR_ITV_5_5_T5_dose'

r='/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/results/'
refim=sitk.ReadImage('../interplay/data/measured/RawImage/1.mhd')
gateim=sitk.ReadImage(r+name+'/'+name+'.mhd')


# get pixels arrays of images and vizualization
gatepix=sitk.GetArrayFromImage(gateim)
refpix=sitk.GetArrayFromImage(refim)                                          
#ct_viewer(refpix)
ct_viewer(gatepix)

interactive(children=(IntSlider(value=22, description='myslice', max=45), Output()), _dom_classes=('widget-int…

In [86]:
# compute true value gate output offset in the DICOM frame
new_origin=np.zeros(3)
#print(gateim.GetOrigin())
new_origin[0]=gateim.GetOrigin()[0]
new_origin[1]=gateim.GetOrigin()[1]*-1
new_origin[2]=gateim.GetOrigin()[2]*-1
gateim.SetOrigin(new_origin)
#print('New Origin : ', gateim.GetOrigin())

#Create a boolean mask to apply on Gate image (remove dose outside the patient according to measurment)
#mask=np.ma.make_mask(refpix)
#apply on gate pixel array
#mask=np.ones_like(refpix)
mask=np.ones_like(gatepix)


gatepix_mask=gatepix*mask
#ct_viewer(gatepix_mask)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#FACTEUR A TROUVER POUR LES PHOTONS !!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# ************ F for interplay ************
E=rp.BeamSequence[0].ControlPointSequence[0].NominalBeamEnergy
mode=rp.BeamSequence[0].PrimaryFluenceModeSequence[0].FluenceMode
energy=str(E)+mode
#energy="6STANDARD"
#print(mode)
if energy == "6NON_STANDARD":
    Dose_Gate_ref=0.0005063408
    N_part_ref=130170012500.0

    
if energy == "6STANDARD":          #avec les killboxes
    Dose_Gate_ref=0.00033116873      #0.00033096725  
    N_part_ref=127666770000.0
    
#nb de part pour faire le phsp réduit
N_part_simu=2606532517
    
print(energy)

#Absolute dose conversion
um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
#fraction unique for interplay
nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)

print('um:', um)
print('frac:', nFrac)



gatepix_abs=(gatepix_mask/N_part_simu)*(0.01)*(N_part_ref/Dose_Gate_ref)*um/100 #*nFrac #un seule fraction here
#gatepix_abs=gatepix_abs/3.55 #scaling factor no interplay F = nb phantom position/100
print("max avant calib:", gatepix_mask.max())
print('maximum:', gatepix_abs.max())

#Create a raw image of gate with mask and a raw image from resampled rt dose dicom 
gate_dose=sitk.GetImageFromArray(gatepix_abs)
gate_dose.SetOrigin(gateim.GetOrigin())
gate_dose.SetSpacing(gateim.GetSpacing())
sitk.WriteImage(gate_dose, r+name+'/'+name+'_abs.mhd')
print('ok')

6NON_STANDARD
um: 623.856997334957
frac: 1.0
max avant calib: 0.0003481591
maximum: 2.1422378707163765
ok


# Absolute dose conversion 

In [146]:
#for isodose value
#print(0.95*gatepix_abs.max())
#print(0.80*gatepix_abs.max())
#print(0.60*gatepix_abs.max())
#print(0.50*gatepix_abs.max())
#print(0.25*gatepix_abs.max())
#print(0.15*gatepix_abs.max())

In [87]:
#Create a raw image of gate with mask and a raw image from resampled rt dose dicom 
gate_dose=sitk.GetImageFromArray(gatepix_abs)
gate_dose.SetOrigin(gateim.GetOrigin())
gate_dose.SetSpacing(gateim.GetSpacing())
sitk.WriteImage(gate_dose, r+name+'/'+name+'_abs.mhd')
print('ok')

ok
